In [0]:
import os

!pip install --upgrade tables
!pip install eli5
!pip install xgboost

os.kill(os.getpid(), 9)

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_cars"

In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values
 

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [9]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [13]:
model = RandomForestRegressor(max_depth=5,n_estimators=50, random_state=0 )
run_model(model, cat_feats)

(-18258.675958165684, 257.4011015428251)

## XGBoost

In [10]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50, 
    'learning_rate' : 0.1,
    'seed' : 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[14:05:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:06:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:06:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [18]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1,seed=0)
m.fit(x, y)

imp = PermutationImportance(m, random_state=0).fit(x, y)
eli5.show_weights(imp, feature_names=cat_feats)

[11:11:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1054 ± 0.0021,param_napęd__cat
0.0917 ± 0.0044,price_value__cat
0.0804 ± 0.0017,param_stan__cat
0.0721 ± 0.0020,param_rok-produkcji__cat
0.0556 ± 0.0011,param_skrzynia-biegów__cat
0.0431 ± 0.0006,param_moc__cat
0.0355 ± 0.0010,param_faktura-vat__cat
0.0225 ± 0.0004,feature_kamera-cofania__cat
0.0213 ± 0.0005,param_marka-pojazdu__cat
0.0190 ± 0.0008,param_typ__cat


In [0]:
feats = [
  'param_napęd__cat',
  'param_rok-produkcji__cat',
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc__cat',
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa__cat',
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',
  'feature_regulowane-zawieszenie__cat',
  'feature_światła-led__cat',
  'param_uszkodzony__cat',
  'feature_klimatyzacja-czterostrefowa__cat',
  'feature_hud-(wyświetlacz-przezierny)__cat',
  'param_rodzaj-paliwa__cat',
  'param_liczba-miejsc',
]

In [15]:
run_model(model, feats)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: '218 KM'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: '90 KM'

  FitFailedWarning)


(nan, nan)

In [29]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [30]:
df['param_rok-produkcji__cat'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, -1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75])

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [32]:
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [45]:
feats = ['param_napęd__cat','price_value__cat','param_stan__cat','param_rok-produkcji','param_skrzynia-biegów__cat','param_moc__cat','param_faktura-vat__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_wersja__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat']
run_model(model, feats)

[11:56:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:56:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:56:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11249.527875659092, 90.69614195639038)

In [34]:
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [17]:
df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0].replace(' ','')) )

0          90
2         115
3         262
4         110
5         310
         ... 
160609     75
160610     75
160611    120
160614    150
160615     88
Name: param_moc, Length: 106494, dtype: int64

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )

In [46]:
feats = ['param_napęd__cat','price_value__cat','param_stan__cat','param_rok-produkcji','param_skrzynia-biegów__cat','param_moc','param_faktura-vat__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_wersja__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat']
run_model( model , feats)

[11:57:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:57:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:57:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9719.708842036986, 73.58446937647294)

In [39]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')) )

In [20]:
feats = [
  'param_napęd__cat',
  'param_rok-produkcji', # normalised
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc', # normalised
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa', # normalised
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',
  'feature_regulowane-zawieszenie__cat',
  'feature_światła-led__cat',
  'param_uszkodzony__cat',
  'feature_klimatyzacja-czterostrefowa__cat',
  'feature_hud-(wyświetlacz-przezierny)__cat',
  'param_rodzaj-paliwa__cat',
  'param_liczba-miejsc',
]

run_model(model, feats)

[14:14:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:14:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:14:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9160.322194698221, 66.16782352644901)